# `ecco` static analysis

In this notebook we see how a `rr` model can be analysed without resorting to the computation of its state-space.

First, we start `ecco` and load our termites `rr` model:

In [1]:
%run -m ecco termites.rr

## Variables characterization

A variable is called _characterized_ if it appears on both sides of some rules. If it appears always on the same side of rules, it is called _semi-characterized_. If it does not appear in rules, then the variable is not used at all and could be removed. Variables characterization may be useful as a sanity check of a large model as a semi-characterized variables has no influence on the model's behaviour, which may contradict the reason why it was introduced in the first place.

Variable characterization can be computed as a matrix using method `model.charact`:

In [4]:
model.charact()

,init,left_0,left_1,right_0,right_1
Ac,1,0,1,0,0
Ec,0,0,1,1,2
Fg,0,1,0,1,1
Rp,1,1,2,1,1
Sd,0,1,0,1,1
Te,0,0,1,2,1
Wd,0,1,1,1,1
Wk,0,2,3,2,1


In this matrix, the rows are indexed by the variables and the columns are has follows:

 * `init` is the initial state of the variable
 * `left_0` counts how many time the variable appears as `-` in the left-hand side of a rule
 * `left_1` counts how many time the variable appears as `+` in the left-hand side of a rule
 * `right_0` counts how many time the variable appears as `-` in the right-hand side of a rule
 * `right_1` counts how many time the variable appears as `+` in the right-hand side of a rule

We can see here that `Ac` is semi-characterized. So it has no influence on the model's beaviour and could be removed, however, it serves as an explanation for the meaning of rule R9 `Ac+, Sd- >> Wk-, Rp-`: without `Ac+` in this rule, one could think that the absence of soldiers may be the cause of the death of the other termites. So, if a semi-characterized variable is not involved in the dynamics, it may have a crucial role for the understanding of the model. Note also that it will not introduce extra combinatorial that could make the state-space grow artificially.

Method `draw_charact` uses the matrix returned by `charact` to display a plot bar of the nodes characterisation:

In [7]:
model.draw_charact()

## Inlining constraints

Method `model.inline` allows to suppress constraints by inlining them into the rules. The idea is that each rule that may enable the execution of a constraint to be inlined is augmented with the right-hand side of the constraint, and possibly with its left-hand side if the rule's condition is not larger that the constraint's condition. Because of the possible augmentation of the left-hand side, a rule may be transformed into several: one that fully validates the condition of the constraint, and (potentially) several that do not validate it. For instance, is we have a constraint is `A+, B+, C+ >> X+` and a rule `A+ >> Z+`, then the rule may yield a state with `A+` that potentially validates the constraints condition. Thus, it will be converted into several rules:

```
A+, B+, C+ >> X+, Z+  # constraint enabled and thus inlined
A+, B- >> X+          # constraint not enabled because of B
A+, C- >> X+          # constraint not enabled because of C
```

In our termites model, inlining constraint C1 just results in replacing rule R7 which is the only one that creates the condition to execute C1. We can proceed as follows:

In [10]:
inlined = model.inline("termites-inlined.rr", "C1", overwrite=True)
inlined.rr

HTML(value="<pre style='line-height:140%'><span style='color:#008; font-weight:bold;'>inhabitants:</span>\n   …

The arguments to `model.inline` are: a file name where the new `rr` model has to be saved, a series of constraints to be inlined, and optionally `overwrite=True` to force overwriting the saved `rr` file if it exists already.

Another method is `model.save` that expects a path to save a `rr` model, with an optional Boolean argument to force overwriting existing files.

### Zeno executions and constraints inlining

Finally, a model may be analyzed to detect its Zeno executions. This is not really done with static analysis as the state-space has to be computed symbolically. But this results in a diagnostic that does not allow to go further into dynamic exploration of the model. So we qualify it as a semi-dynamic method.

A model is called Zeno if it has cyclic executions involving only constraints. This means that it will potentially stay forever into transient states, which somehow contradicts the intuition about transient states. However, there may be models in which Zeno executions can be considered as perfectly valid executions. In order to search for such executions, we can run method `model.zeno`:

In [15]:
model.zeno()

HTML(value='<p style="line-height:140%"><span style="color:#008800; font-weight:bold;">[info]</span> initial s…

It will print a diagnosis (here, the model is non-Zeno) and, if the model is found to be Zeno, witness cycles are displayed. An optional integer argument can be passed to control the maximum number of witnesses that will be returned (and displayed). When a model is Zeno, one way to change this is to inline the constraints that participate to Zeno cycles.